# Onboarding Agent - Your AI-Powered Onboarding Buddy

Welcome to the Onboarding Agent workshop! In this notebook, you'll learn how to create an intelligent AI agent that serves as an onboarding buddy for new hires, helping them navigate their first days at a company with personalized guidance and support.

## What You'll Learn

By the end of this notebook, you'll understand:

1. **Onboarding Agent Architecture** - How to structure an agent that manages complex onboarding workflows
2. **Task Management** - Creating plugins that organize and present onboarding tasks systematically
3. **Document and Reference Management** - How agents can provide contextual help with company resources
4. **Progressive Learning** - Building agents that adapt to the employee's progress and role
5. **Multi-Phase Onboarding** - Organizing tasks into logical phases (Paperwork, Training, Content)

## What is an Onboarding Agent?

An Onboarding Agent is a specialized AI assistant designed to:

- **Guide new employees** through their onboarding journey step-by-step
- **Answer questions** about company policies, procedures, and culture
- **Organize tasks** into manageable phases and priorities
- **Provide contextual help** based on the employee's role and department
- **Track progress** and suggest next steps in the onboarding process
- **Connect employees** with relevant resources, documents, and people

## The Onboarding Buddy Example

In this notebook, we'll build an **Onboarding Buddy** that can:
- Present onboarding tasks in an organized, prioritized manner
- Answer role-specific questions about the company and position
- Provide guidance on paperwork, training, and learning content
- Adapt responses based on the employee's progress and needs
- Maintain a helpful, encouraging, and supportive tone

This demonstrates how AI agents can transform the employee experience by providing personalized, always-available support during the critical early days of employment.

## Import the Required Python Packages

We'll use the same core Semantic Kernel components as other workshops, but organized for onboarding-specific functionality:

### Core Python Libraries
- `os` - For accessing environment variables and file operations
- `json` - For handling structured onboarding data and task definitions
- `datetime` - For tracking onboarding timelines and deadlines
- `typing.Annotated` - For type hints that improve function descriptions

### Azure Authentication
- `azure.identity.DefaultAzureCredential` - For secure Azure AD authentication
- `dotenv.load_dotenv` - For loading environment variables from .env files

### Jupyter Notebook Display
- `IPython.display` - For rich HTML output showing organized onboarding content

### Semantic Kernel Components
- `ChatCompletionAgent` - The main onboarding agent class
- `ChatHistoryAgentThread` - Manages conversation history across onboarding sessions
- `AzureChatCompletion` - Connector for Azure OpenAI services
- `kernel_function` - Decorator for creating onboarding-specific functions (tools)

In [ ]:
# Install required packages from requirements.txt
%pip install -r requirements.txt

In [ ]:
import os 
import json
from datetime import datetime, timedelta
from typing import Annotated
from azure.identity import DefaultAzureCredential

from dotenv import load_dotenv

from IPython.display import display, HTML

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## Creating the Onboarding Plugin

The OnboardingPlugin is the heart of our agent's functionality. It manages three key areas of the onboarding experience:

### 1. Task Management
- **Paperwork Tasks**: Legal documents, benefits enrollment, tax forms
- **Training Tasks**: Mandatory training modules, role-specific learning
- **Content Tasks**: Company culture, team introductions, resource familiarization

### 2. Document and Reference Management
- Company handbook and policies
- Role-specific resources and documentation
- Team contact information and organizational charts
- Learning materials and training content

### 3. Progress Tracking
- Completion status of various onboarding phases
- Upcoming deadlines and priorities
- Personalized recommendations based on role and department

### Plugin Design Principles

Our plugin demonstrates several best practices:

- **Structured Data**: Tasks are organized into clear categories and phases
- **Role-Based Customization**: Different content for different positions
- **Progressive Disclosure**: Information is presented in digestible chunks
- **Contextual Help**: Responses adapt to where the employee is in their journey
- **Action-Oriented**: Every interaction guides the employee toward next steps

In [ ]:
class OnboardingPlugin:
    """An AI-powered onboarding buddy that guides new employees through their onboarding journey."""

    def __init__(self):
        # Simulated onboarding data - in a real implementation, this would connect to HR systems
        self.onboarding_tasks = {
            "paperwork": [
                {"task": "Complete I-9 Employment Eligibility Verification", "priority": "High", "deadline": "Day 1", "status": "pending"},
                {"task": "Submit tax withholding forms (W-4)", "priority": "High", "deadline": "Day 1", "status": "pending"},
                {"task": "Enroll in health insurance benefits", "priority": "High", "deadline": "Day 3", "status": "pending"},
                {"task": "Set up direct deposit for payroll", "priority": "Medium", "deadline": "Day 5", "status": "pending"},
                {"task": "Complete emergency contact information", "priority": "Medium", "deadline": "Day 3", "status": "pending"},
                {"task": "Review and sign employee handbook acknowledgment", "priority": "Medium", "deadline": "Day 5", "status": "pending"}
            ],
            "training": [
                {"task": "Complete mandatory workplace safety training", "priority": "High", "deadline": "Week 1", "status": "pending"},
                {"task": "Attend new employee orientation session", "priority": "High", "deadline": "Day 2", "status": "pending"},
                {"task": "Complete cybersecurity awareness training", "priority": "High", "deadline": "Week 1", "status": "pending"},
                {"task": "Take diversity and inclusion training", "priority": "Medium", "deadline": "Week 2", "status": "pending"},
                {"task": "Complete role-specific technical training", "priority": "High", "deadline": "Week 2", "status": "pending"},
                {"task": "Shadow team members for hands-on learning", "priority": "Medium", "deadline": "Week 3", "status": "pending"}
            ],
            "content": [
                {"task": "Read company mission, vision, and values", "priority": "Medium", "deadline": "Week 1", "status": "pending"},
                {"task": "Review organizational chart and team structure", "priority": "Medium", "deadline": "Day 3", "status": "pending"},
                {"task": "Explore company intranet and resources", "priority": "Low", "deadline": "Week 1", "status": "pending"},
                {"task": "Schedule meet-and-greets with key team members", "priority": "High", "deadline": "Week 1", "status": "pending"},
                {"task": "Join relevant Slack channels and teams", "priority": "Medium", "deadline": "Day 2", "status": "pending"},
                {"task": "Review department-specific documentation", "priority": "Medium", "deadline": "Week 2", "status": "pending"}
            ]
        }

        self.company_info = {
            "name": "TechCorp Innovation Labs",
            "mission": "Empowering businesses through cutting-edge technology solutions",
            "values": ["Innovation", "Integrity", "Collaboration", "Customer Focus", "Excellence"],
            "departments": ["Engineering", "Product", "Sales", "Marketing", "HR", "Finance"],
            "benefits": ["Health Insurance", "401k Matching", "Flexible PTO", "Professional Development", "Remote Work Options"],
            "resources": {
                "hr_contact": "hr@techcorp.com",
                "it_helpdesk": "support@techcorp.com", 
                "employee_handbook": "https://intranet.techcorp.com/handbook",
                "benefits_portal": "https://benefits.techcorp.com",
                "learning_platform": "https://learning.techcorp.com"
            }
        }

    @kernel_function(description="Gets organized onboarding tasks by category (paperwork, training, or content).")
    def get_onboarding_tasks(
        self, 
        category: Annotated[str, "The category of tasks to retrieve: 'paperwork', 'training', 'content', or 'all'"] = "all"
    ) -> Annotated[str, "Returns organized onboarding tasks with priorities and deadlines."]:
        
        if category.lower() == "all":
            all_tasks = []
            for cat, tasks in self.onboarding_tasks.items():
                all_tasks.append(f"\n🎯 **{cat.upper()} TASKS:**")
                for i, task in enumerate(tasks, 1):
                    status_emoji = "✅" if task["status"] == "completed" else "⏳"
                    priority_emoji = "🔴" if task["priority"] == "High" else "🟡" if task["priority"] == "Medium" else "🟢"
                    all_tasks.append(f"{i}. {status_emoji} {priority_emoji} {task['task']}")
                    all_tasks.append(f"   📅 Deadline: {task['deadline']} | Priority: {task['priority']}")
            return "\n".join(all_tasks)
        
        elif category.lower() in self.onboarding_tasks:
            tasks = self.onboarding_tasks[category.lower()]
            result = [f"\n🎯 **{category.upper()} TASKS:**"]
            for i, task in enumerate(tasks, 1):
                status_emoji = "✅" if task["status"] == "completed" else "⏳"
                priority_emoji = "🔴" if task["priority"] == "High" else "🟡" if task["priority"] == "Medium" else "🟢"
                result.append(f"{i}. {status_emoji} {priority_emoji} {task['task']}")
                result.append(f"   📅 Deadline: {task['deadline']} | Priority: {task['priority']}")
            return "\n".join(result)
        
        else:
            return f"❌ Category '{category}' not found. Available categories: paperwork, training, content, or all"

    @kernel_function(description="Provides company information, policies, and resources for new employees.")
    def get_company_information(
        self, 
        info_type: Annotated[str, "Type of information: 'overview', 'values', 'benefits', 'contacts', 'resources', or 'all'"] = "overview"
    ) -> Annotated[str, "Returns relevant company information for onboarding."]:
        
        if info_type.lower() == "overview" or info_type.lower() == "all":
            overview = [
                f"🏢 **Welcome to {self.company_info['name']}!**",
                f"\n🎯 **Our Mission:** {self.company_info['mission']}",
                f"\n⭐ **Our Core Values:**"
            ]
            for value in self.company_info['values']:
                overview.append(f"   • {value}")
            
            overview.append(f"\n🏛️ **Departments:** {', '.join(self.company_info['departments'])}")
            
            if info_type.lower() == "all":
                overview.extend([
                    f"\n🎁 **Benefits Package:**"
                ])
                for benefit in self.company_info['benefits']:
                    overview.append(f"   • {benefit}")
                    
                overview.extend([
                    f"\n📞 **Key Contacts:**",
                    f"   • HR Support: {self.company_info['resources']['hr_contact']}",
                    f"   • IT Helpdesk: {self.company_info['resources']['it_helpdesk']}",
                    f"\n🔗 **Important Resources:**",
                    f"   • Employee Handbook: {self.company_info['resources']['employee_handbook']}",
                    f"   • Benefits Portal: {self.company_info['resources']['benefits_portal']}",
                    f"   • Learning Platform: {self.company_info['resources']['learning_platform']}"
                ])
            
            return "\n".join(overview)
        
        elif info_type.lower() == "benefits":
            benefits = [f"🎁 **Benefits Package at {self.company_info['name']}:**"]
            for benefit in self.company_info['benefits']:
                benefits.append(f"   • {benefit}")
            benefits.append(f"\n💡 Access your benefits portal: {self.company_info['resources']['benefits_portal']}")
            return "\n".join(benefits)
        
        elif info_type.lower() == "contacts":
            contacts = [
                "📞 **Key Contacts for New Employees:**",
                f"   • HR Support: {self.company_info['resources']['hr_contact']}",
                f"   • IT Helpdesk: {self.company_info['resources']['it_helpdesk']}",
                f"\n💡 Don't hesitate to reach out if you need any assistance!"
            ]
            return "\n".join(contacts)
        
        elif info_type.lower() == "resources":
            resources = [
                "🔗 **Essential Resources for Your Success:**",
                f"   • Employee Handbook: {self.company_info['resources']['employee_handbook']}",
                f"   • Benefits Portal: {self.company_info['resources']['benefits_portal']}",
                f"   • Learning Platform: {self.company_info['resources']['learning_platform']}",
                f"\n💡 Bookmark these links for easy access!"
            ]
            return "\n".join(resources)
        
        elif info_type.lower() == "values":
            values = [f"⭐ **Core Values at {self.company_info['name']}:**"]
            for value in self.company_info['values']:
                values.append(f"   • {value}")
            values.append(f"\n💡 These values guide everything we do and how we work together!")
            return "\n".join(values)
        
        else:
            return f"❌ Information type '{info_type}' not available. Try: overview, values, benefits, contacts, resources, or all"

    @kernel_function(description="Provides personalized onboarding guidance and next steps based on the employee's current progress.")
    def get_personalized_guidance(
        self, 
        employee_role: Annotated[str, "The employee's role or department"] = "general",
        current_day: Annotated[str, "Current day in onboarding process (e.g., 'Day 1', 'Week 1')"] = "Day 1"
    ) -> Annotated[str, "Returns personalized guidance and recommended next steps."]:
        
        guidance = [
            f"🌟 **Personalized Onboarding Guidance for {employee_role} - {current_day}**",
            f"\n👋 Great to have you on the team! Here's what I recommend focusing on right now:"
        ]
        
        # Day-specific guidance
        if "Day 1" in current_day or "day 1" in current_day.lower():
            guidance.extend([
                "\n🎯 **Today's Priorities:**",
                "   1. Complete I-9 and tax forms (HR will guide you)",
                "   2. Set up your workspace and computer access",
                "   3. Join the new employee orientation session",
                "   4. Meet your direct manager and team",
                "   5. Review emergency contact requirements"
            ])
        elif "Day 2" in current_day or "day 2" in current_day.lower():
            guidance.extend([
                "\n🎯 **Today's Focus:**",
                "   1. Join relevant communication channels (Slack, Teams)",
                "   2. Complete cybersecurity training",
                "   3. Review organizational chart",
                "   4. Schedule one-on-ones with key team members",
                "   5. Explore the company intranet"
            ])
        elif "Week 1" in current_day or "week 1" in current_day.lower():
            guidance.extend([
                "\n🎯 **This Week's Goals:**",
                "   1. Complete all mandatory training modules",
                "   2. Finish benefits enrollment",
                "   3. Familiarize yourself with company policies",
                "   4. Connect with cross-functional partners",
                "   5. Set up development/learning goals"
            ])
        
        # Role-specific guidance
        if "engineer" in employee_role.lower() or "developer" in employee_role.lower():
            guidance.extend([
                "\n💻 **Engineering-Specific Tasks:**",
                "   • Set up development environment and tools",
                "   • Review code standards and deployment processes",
                "   • Join engineering Slack channels and stand-ups",
                "   • Schedule architecture overview session"
            ])
        elif "sales" in employee_role.lower():
            guidance.extend([
                "\n💼 **Sales-Specific Tasks:**",
                "   • Access CRM system and sales tools",
                "   • Review current product offerings and pricing",
                "   • Shadow experienced sales team members",
                "   • Learn about target customers and markets"
            ])
        elif "marketing" in employee_role.lower():
            guidance.extend([
                "\n📈 **Marketing-Specific Tasks:**",
                "   • Access marketing automation tools",
                "   • Review brand guidelines and messaging",
                "   • Join marketing campaign planning sessions",
                "   • Understand customer personas and journey"
            ])
        
        guidance.extend([
            "\n💡 **Remember:** Take your time, ask questions, and don't hesitate to reach out for help!",
            f"📞 Need immediate assistance? Contact HR at {self.company_info['resources']['hr_contact']}"
        ])
        
        return "\n".join(guidance)

## Creating the AI Service Connection

Just like in our other workshops, we need to establish a connection to Azure OpenAI. The onboarding agent uses the same robust, enterprise-grade AI service to provide intelligent, contextual responses.

### Configuration for Onboarding Scenarios

Our onboarding agent will use these AI capabilities:

- **Natural Language Understanding** - Interprets employee questions about policies, procedures, and company culture
- **Task Prioritization** - Helps employees understand what to focus on first based on their role and timeline
- **Contextual Responses** - Provides relevant information based on the employee's current onboarding phase
- **Conversational Memory** - Remembers previous interactions to provide continuity across onboarding sessions

### Authentication Best Practices

For onboarding systems handling sensitive employee data:

- **Secure Credential Management** - Never hard-code API keys or sensitive information
- **Environment Variables** - Store credentials securely in environment variables or key vaults
- **Azure AD Integration** - Use Azure AD for enterprise authentication when possible
- **Principle of Least Privilege** - Grant only the minimum permissions necessary

💡 **Production Tip**: In a real onboarding system, you'd integrate with HR systems like Workday, BambooHR, or ADP to pull actual employee data and task status.

In [ ]:
load_dotenv()

# Option 1: Using API Key (recommended for development)
chat_completion_service = AzureChatCompletion(
    deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
    endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY")
)

# Option 2: Using Azure AD Authentication (recommended for production)
# credential = DefaultAzureCredential()

# def get_azure_ad_token():
#     """Function to get Azure AD token for OpenAI."""
#     token = credential.get_token("https://cognitiveservices.azure.com/.default")
#     return token.token

# chat_completion_service = AzureChatCompletion(
#     deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
#     endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
#     api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
#     ad_token=get_azure_ad_token()
# )

## Creating the Onboarding Agent

Now we'll create our **Onboarding Buddy** using carefully crafted instructions that ensure a supportive, organized, and helpful experience for new employees.

You can change these settings to see how the differences in the agent's response.

In [ ]:
# Create the agent
agent = ChatCompletionAgent(
    service=chat_completion_service,
    name="OnboardingBuddy",
    instructions="You are Joule, a friendly AI-powered onboarding buddy at TechCorp Innovation Labs. Help new employees navigate their onboarding journey by organizing tasks into Paperwork, Training, and Content phases. Provide personalized guidance based on their role and current onboarding phase. Be warm, welcoming, and supportive while keeping information organized and actionable.",
    plugins=[OnboardingPlugin()],
)

## Running the Agent 

Now we will run the AI Agent. In this snippet, we can add multiple messages to the `user_input` to show how the agent responds to different onboarding scenarios.

The agent should call the correct function to get onboarding tasks, company information, and provide personalized guidance based on the employee's role and current phase.

You can change the `user_inputs` to see how the agent responds.

In [ ]:
user_inputs = [
    "Hi! I'm a new software engineer starting today. Can you help me get oriented?",
    "Can you show me all my onboarding tasks?",
    "I'm a software engineer on Day 2. What should I prioritize today?",
    "Can you tell me about the company culture and values?"
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()